# Single-Cell RNAseq Analysis: Preprocessing and Cell Type Clustering

In this project you will acquire hands-on experience analysing real-world single-nucleus RNA sequencing (snRNA-seq) data. The goal of this introductory exercise is to get familiarity with Python syntax, data types and structures, and grow your practical skills in large-scale data manipulation and visualisation through an analysis of cell clustering. From an applied standpoint, you’ll be introduced to powerful Python packages commonly used in the analysis of single-cell gene expression data.

__Workflow__

This notebook will guide you through the basic analysis steps required to conduct cell-type clustering of snRNA-seq data. A step-by-step outline is provided below.

_Data reading and exploration_ 
This notebook starts from a preprocessed AnnData object containing raw expression counts for 38,917 genes across 7,140 cells derived from the kidney of a healthy male. Gene-level and cell(barcode)-level metadata are also stored in this object. The initial step is to locate, read, and explore the data. 

_Filtering of genes and barcodes_  
An essential processing step is the identification and filtering of zero- and lowly-expressed genes, which don’t add much to the transcriptomic heterogeneity of the cells, which we often aim to study.  Equally important is the detection of barcodes with outlier quality-control (QC) metrics, representing either poor-quality cells, empty droplets, or droplets with more than one cell/nucleus; followed by analytical decisions to exclude or retain such barcodes for downstream analysis.  

_Normalisation of gene expression data_  
In this step, raw read counts are adjusted for differences in sequencing depth across cells, enabling the comparison of expression levels between them. Counts are also transformed into a more interpretable scale with convenient statistical properties for downstream applications. Common methods include library size normalisation followed by log transformation.

_Dimensionality reduction_  
We perform Principal Component Analysis (PCA) on the data, which is a common first step to linearly reduce the dimensionality of normalised data.

_Cell clustering_  
Unsupervised cell clustering aims to group the data (cells) based on their transcriptomic similarities in a reduced space. This is done using algorithms such as K-means and Leiden.  Non-linear dimensionality reduction techniques such as UMAP (Uniform Manifold Approximation and Projection) or t-SNE (t-distributed Stochastic Neighbor Embedding) are used for visualising the resulting cell clusters and cell type marker expression.


__What you need__  
You are expected to have at least basic Python knowledge, specifically in data handling and visualisation, a basic understanding of molecular cell biology and genomics.

__What is not required__  
Despite the nature of the analysis, you do not need a background in single-cell/nucleus RNAseq to be able to perform this project, nor a deep understanding of the mathematical theory behind dimensionality reduction and clustering. 

__Instructions__  
Work through this notebook from top to bottom, executing coding cells sequentially.  Answer __Questions__ and perform __Tasks__ you see along the way.  You are expected to write and demonstrate your answers by adding cells of text or code, where appropriate. 

__Authors__  
Iris Diana Yu, EMBL-EBI  
Daianna Gonzalez Padilla, Wellcome Sanger Institute


## Table of contents  
1. [Background](#background)
2. [Reading data](#reading_data)
3. [Gene annotation](#gene_annotation)
4. [Filtering](#filtering)  
    4.1 [Filtering barcodes](#filtering_barcodes)  
    4.2 [Filtering genes](#filtering_genes)  
    4.3 [Doublet removal](#doublet_removal)  
5. [Normalisation](#normalisation)

## 1. Background

### Data

The data analysed in this notebook were generated in [Muto et al. (Nature Communications, 2021)](https://www.nature.com/articles/s41467-021-22368-w) and retrieved from the Single Cell Expression Atlas, accession [E-CURD-119](https://www.ebi.ac.uk/gxa/sc/experiments/E-CURD-119/results/cell-plots), [downloaded here](https://www.ebi.ac.uk/gxa/sc/experiments/E-CURD-119/downloads). In the study, the cellular diversity of the adult human kidney was thoroughly characterized through single-cell transcriptome and chromatin accessibility profiling, analysing kidney samples from five healthy adults.  


In this project, we will only analyse the single-cell transcriptomics data for one sample corresponding to a healthy 54-year old male known as `Healthy1`.  

We start with the partially filtered sampled data set contained in an AnnData `.h5ad` file.

### Setting up

In [ ]:
import pandas as pd
import scanpy as sc
import seaborn as sns

# set plotting theme
sns.set_theme()

<a id="reading_data"></a>
## 2. Reading data

We start by reading the data using one of the read functions from `scanpy` ([documented here](https://scanpy.readthedocs.io/en/stable/api/reading.html)).

In [ ]:
adata = sc.read("project_data.h5ad")
adata

This AnnData object stores several pieces of information, including: 

- A matrix of counts, with samples as rows and genes as columns. This can be retrived with the `.X` accessor.
- Metadata about the barcodes (cells), which can be retrieved using the `.obs` accessor. 
- Metadata about the genes, which can be retrived using the `.var` accessor.

In the output above, the top line `AnnData object with n_obs × n_vars = 7140 × 38917` tells us the shape of the data set.  The `n_obs` is the number of observations in `.obs`--in this case, the sequencing barcodes, which we can more or less think of as equivalent to cells.  The `n_vars` is the number of variables in `.vars`, which in our case are genes. 

![Diagram of the AnnData object type. [Source](https://scanpy.readthedocs.io/en/stable/usage-principles.html)](https://falexwolf.de/img/scanpy/anndata.svg)


### Question
What data types are `adata.X`, `adata.obs`, and `adata.var` respectively?

<a id="gene_annotation"></a>
## 3. Gene Annotation

Let's inspect `adata.var`, where the gene metadata are stored.

In [ ]:
adata.var

Above we see that `adata.var` is indexed with the human ENSEMBL gene IDs.  The human gene IDs can be explored [here](https://www.ensembl.org/Homo_sapiens/Info/Index).  The common gene name equivalent of these gene IDs are stored in `adata.var["gene_name"]`.

### Questions
1. What kind of information do you think are stored in the `adata.var` columns, namely the "gene_symbols", "chromosome", and "mito" columns?
2. What values are stored in the `adata.var` column "mito"? How many unique values are there?
3. What values are stored in the `adata.var` column "chromomosome". How many unique values are there?
4. What is the relationship of values between `adata.var` columns "mito" and "chromosome"?
5. Upon inspecting the `adata.var` column "chromosome", do you notice anything unusual?

We would like to keep only genes in the autosomes (1 through 23), X, Y and MT chromosome (i.e., remove genes in unassembled scaffolds).  Below is how we perform it.

In [ ]:
vars_to_keep = (
  adata.var["chromosome"]
  .isin([str(i) for i in range(1, 23)] + ["X", "Y", "MT"])
)
adata = adata[:, vars_to_keep].copy()
adata

Note the use of the `.copy()` method. This ensures that we make a new copy of the object (which we replace back into `adata`), rather than a "View" of the original object.

### Questions
1. Explain each line of code in the cell that modifies `adata` to retain genes mapping to valid chromosomes. 
2. How many genes do we now have in our data, after retaininig only genes in valid chromosomes?

<a id="filtering"></a>
## 4. Filtering

We start by doing some exploratory analysis of our count data, namely in terms of:  

- number of total counts per barcode  
- number of detected genes per barcode  
- fraction of counts in mitochondrial genes

We use the function `scanpy.pp.calculate_qc_metrics`.  You can read more about the function [in the scanpy documentation](https://scanpy.readthedocs.io/en/stable/generated/scanpy.pp.calculate_qc_metrics.html).

In [ ]:
sc.pp.calculate_qc_metrics(
    adata, 
    qc_vars=["mito"], 
    inplace=True, 
    percent_top=[20], 
    log1p=True
)
adata

### Questions  
1. Explain each parameter that we supplied to `sc.pp.calculate_qc_metrics`.
2. The execution of the function added several metrics to our barcode/cell metadata, i.e. our `adata.obs`.  What are these new metrics?  Explain each.
3. Some metrics were also added to the gene metadata, `adata.var`.  What are these new metrics? Explain each.

<a id="filtering_barcodes"></a>
### 4.1 Filtering barcodes

Since `adata.obs` is a regular Pandas DataFrame, we can use standard plotting libraries to visualise some of the statistics that the previous function calculated for us.  

### Task  
Using Seaborn, generate the following:
1. A plot showing the distribution of total UMI counts per cell. Use 20 bins.
2. A plot showing the distribution of percent mitochondrial UMIs per cell. Use 20 bins.
3. A plot showing the relationship between the total UMI counts per cell and the number of genes with at least 1 count in a cell.  Color the data points based on the percent of mitochondrial UMIs per cell.  

### Question  
Based on each plot, what can you say about the data?

Alternatively, we can use `scanpy`'s own plotting functions (histogram is not available, but we can do violin plots instead):  

In [ ]:
sc.pl.violin(adata, "total_counts")
sc.pl.violin(adata, "pct_counts_mito")
sc.pl.scatter(adata, "total_counts", "n_genes_by_counts", color="pct_counts_mito")

We can filter our barcodes/cells based on hard thresholds set manually.  Alternatively, we can define a function that removes outliers based on the observed distribution.  For instance, we can use an `n` number of median absolute deviations (MAD) to determine outliers. More about MAD in [this Wikipedia entry](https://en.wikipedia.org/wiki/Median_absolute_deviation). Below, we create a MAD-based function that determines outlier observations in a given metric. It uses the `scipy.stats` function `median_abs_deviation`, with documentation [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.median_abs_deviation.html).

In [ ]:
import numpy as np
from scipy.stats import median_abs_deviation

def is_outlier(adata, metric: str, nmads: int):
  M = adata.obs[metric]
  
  outlier = (M < np.median(M) - nmads * median_abs_deviation(M)) | (
      np.median(M) + nmads * median_abs_deviation(M) < M
  )
  return outlier

### Questions  
Explain the function `is_outlier`.  
1. What are the inputs and what is their role in the function?
2. What kind of data is passed onto the variable `outlier`?
3. Explain the expression used to derive `outlier`.

### Tasks  
1. Use the function `is_outlier` to create new barcode metadata columns indicating counts outliers.  Use `nmads = 5`.  Do this for each of the following:
    1. For raw counts outliers, use `counts_outlier` as the new metadata name. Determine outliers using `log1p_total_counts`.
    2. For gene counts outliers, use `genes_outlier` as the new metadata name.  Determine outliers using `log1p_n_genes_by_counts`.
    3. For top genes outliers, use `topgenes_outlier` as the new metadata name.  Determine outliers using `pct_counts_in_top_20_genes`.  

2. Create a histogram for each of the chosen metrics above, and color the histogram by based on outliers.  Hint: `sns.displot(adata.obs, x="metric_col_name", hue="outlier_col_name", multiple="stack")`.  

Note that we choose to use the log-transformed counts, as its distribution should be less skewed and therefore more suitable for MAD-based filtering.  

We also check for outliers with regards to percentage of mitochondrial counts, where we use more strict filters: 

In [ ]:
adata.obs["mito_outlier"] = is_outlier(adata, "pct_counts_mito", 3) | (adata.obs["pct_counts_mito"] > 8)
sns.displot(adata.obs, x="pct_counts_mito", hue="mito_outlier", multiple="stack")

### Question  
1. How many are considered outliers per metric filter?
2. Discuss the data based on what each of the four plots show.

Let us now create a variable which is the union of the conditions we set for our filters.  That is, if a barcode is determined an outlier of _any_ of our filters, then we consider it to be an outlier. 

In [ ]:
adata.obs["outlier"] = adata.obs["genes_outlier"] | adata.obs["genes_outlier"] | adata.obs["topgenes_outlier"] | adata.obs["mito_outlier"]
adata.obs["outlier"].value_counts()

### Task  
Create the same scatterplot of counts vs detected genes you created earlier, but color the data points according which barcodes will be removed. 

We now remove all outliers from our data set.

In [ ]:
adata = adata[~adata.obs["outlier"], :].copy()
adata

### Question  

The barcodes we are left with after this filtering we will now consider to be cells. How many cells are we left with?

<a id="filtering_genes"></a>
### 4.2 Filtering Genes

In the same way that we explored several metrics for barcodes, we can also explore them for genes. However, as we will see, downstream analysis can focus on variable genes and will mostly ignore genes for which there is very little data. Therefore, we don't perform as strict filtering on genes as we do on barcodes.    

Still, it is useful to remove undetected genes, i.e. those with zero total counts:

### Task  
Count how many genes have zero counts.  Hint: You may use the function `pd.DataFrame.eq()` function, [documentation here](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.eq.html).

There is an out-of-the box method in `scanpy` to filter genes--`sc.pp.filter_genes()`.  The document is [here](https://scanpy.readthedocs.io/en/stable/generated/scanpy.pp.filter_genes.html).  We do the filtering as below.

In [ ]:
sc.pp.filter_genes(adata, min_counts=0)

### Task  
Check number how many genes are left after removing zero-count genes.adata.n_vars

<a id="doublet_removal"></a>
### 4.3 Doublet removal

In this special barcode filtering step, we remove droplet barcodes that may contain more than one cell or nucleus. This is an important step because, unremoved, these droplets known as "doublets" can be misclassified and thus confound downstream analysis.

### Question  
The `adata.obs` column `predicted_doublet` stores information on which cells are predicted doublets.  How many are predicted doublets and how many are not? 

At the moment, our count matrix has the following shape.

In [ ]:
adata.shape

Below is how we would filter out predicted doublets.

In [ ]:
# adata = adata[~adata.obs["predicted_doublet"], :].copy()

<a id="normalisation"></a>
## 5. Normalisation

There are several normalisation methods available for single-cell RNA-seq data. In this exercise, we will use shifted logarithm method.  This methods scales the counts by a cell-specific size factor (based on the total counts in that cell) followed by taking its logarithm. Despite being a relatively simple method, it has been shown to perform well in downstream analysis such as dimensionality reduction and clustering.  

We will use the `layers` component of the AnnData object, which can be used to store different versions of our count matrix.  This is a good way to keep several versions of our data in place, especially as we explore different methods of normalisation.

In [ ]:
from scipy.sparse import csr_matrix

# keep a copy of the raw counts in the object as a backup
adata.layers["counts"] = adata.X.copy()

# create new layer for log-normalised counts
adata.layers["logcounts"] = adata.X.copy()
sc.pp.normalize_total(adata, layer="logcounts", target_sum=10000)
sc.pp.log1p(adata, layer="logcounts")

### Question  
1. For normalisation, what does each line of the code do?  Refer to the documentations of the functions.
2. In what way did `adata` change after we performed normalisation?
3. What type of variable was introduced in the layer that was created to contain logcounts?

### Task  
Visualise the normalised data.

We now save our processed data.

In [ ]:
adata.write(f"project_data_processed.h5ad")

## We stop here for Day 1 of the Group Project 

## We start here again for Day 2 of the Group Project

<a id="dimred"></a>
## 6. Dimensionality reduction

<a id="clustering"></a>
## 7. Clustering